In [1]:
import pathlib
import scipy.spatial.distance as ssd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Single_Set collects all the results for Strainr

In [293]:

def single_set(species_name,num_strains):
        """ This takes the name of 2 folders, and calculates the 10 output files compared to their true values"""
        distlist = []
        val_list=[]
        bval_list=[]
        for i in range(1,11):
                # make filename
                aout = 'run_' + str(i) + '_abundance.txt'
                sout =  'run_' + str(i) +  '_R1.fastq' 
                true_abundance_file = base / species_name / num_strains / aout
                exp_abundance_file = base / species_name / num_strains / 'results' / sout / 'intra_abundance.tsv'

                # get filevals
                vals,bvals = [],[]
                for line in true_abundance_file.open():
                        vals.append(float(line.split()[1]))
                for line in exp_abundance_file.open():
                        bvals.append(float(line.split()[-1]))

                # wrangle lists
                vals.sort(reverse=True)
                bvals.sort(reverse=True)
                bvals = bvals[:len(vals)]
                X = np.array(list(zip(bvals,vals))).T

                # calculate dist
                dist = (ssd.pdist(X,'cityblock'))
                distlist.append(dist[0])
                val_list.append(vals)
                bval_list.append(bvals)

        return np.array(distlist),val_list,bval_list



# Here we call the function in a loop for each species, and plot the results

In [294]:
import matplotlib.ticker as ticker
sns.set_style('whitegrid')
base = pathlib.Path('/hdd/compare_strainr')
strains = ['Streptococcus_Pneumoniae','Parabacteroides_Distasonis','Haemophilus_Influenzae','Escherichia_Coli','Staph_Aureus']
nstrains = ['2strain','3strain','4strain']
set_of_sets2= {}
set_of_sets3= {}
set_of_sets4= {}
vlist=[]
blist=[]
for s in strains[:]:
        for n in nstrains[0:1]:
                a,b,c = single_set(s,n)
                set_of_sets2[s] = a
                vlist.append(b)
                blist.append(c)
for s in strains[:]:
        for n in nstrains[1:2]:
                a,b,c = single_set(s,n)
                set_of_sets3[s] = a
                vlist.append(b)
                blist.append(c)
for s in strains[:]:
        for n in nstrains[2:3]:
                a,b,c = single_set(s,n)
                set_of_sets4[s] = a
                vlist.append(b)
                blist.append(c)


df2 = pd.DataFrame(set_of_sets2)
df3 = pd.DataFrame(set_of_sets3)
df4 = pd.DataFrame(set_of_sets4)
# fig = plt.figure(figsize=(8,5))
# ax = fig.add_subplot(111)
# g = sns.boxplot(data=df)
# g.set(ylim=(0,1))
# g.tick_params(rotation=90)
# g.xaxis.set_major_locator(ticker.MultipleLocator(1))
# g.set(title='Jensen-Shannon Distance by Species',xlabel='species',ylabel='JSD')
df

,Streptococcus_Pneumoniae,Parabacteroides_Distasonis,Haemophilus_Influenzae,Escherichia_Coli,Staph_Aureus,method
0,0.221158,0.019893,0.007226,0.298280,0.017672,Strainr
1,0.068972,0.055138,0.005047,0.018457,0.384680,Strainr
2,0.194116,0.025850,0.004710,0.374510,0.044230,Strainr
3,0.017321,0.182216,0.009227,0.075014,0.834538,Strainr
4,0.397907,0.044258,0.017370,0.459076,0.035669,Strainr
5,0.120470,0.006842,0.003856,0.487260,0.297967,Strainr
6,0.047673,0.008647,0.012189,0.204260,0.381450,Strainr
7,0.075508,0.255046,0.025945,0.005186,0.066988,Strainr
8,0.119500,0.107021,0.001403,0.293690,0.213950,Strainr
9,0.116649,0.014994,0.068064,0.493799,0.186647,Strainr


In [295]:
df2['nstrain'] = 2
df3['nstrain'] = 3
df4['nstrain'] = 4
df23 = pd.concat([df2,df3,df4])

# Then we do something similiar - Kraken2 vs the true abundance

In [296]:

def single_krakenset(species_name,num_strains):
        """ This takes the name of 2 folders, and calculates the 10 output files compared to their true values"""
        distlist = []
        val_list=[]
        kval_list=[]
        for i in range(1,11):
                # make filename
                aout = 'run_' + str(i) + '_abundance.txt'
                sout =  'report' + str(i) +  '.tsv' 
                true_abundance_file = base / species_name / num_strains / aout
                kraken_abundance_file = base / species_name / num_strains / 'kraken2_results' / sout


                # get true abundance 
                vals = []
                for line in true_abundance_file.open():
                        vals.append(float(line.split()[1]))
                vals.sort(reverse=True)

                # Get kraken values
                df = pd.read_csv(kraken_abundance_file, sep="\t", header=None)
                df.columns = ['relab','count','count2','taxrank','taxid','name']
                df = df[df.taxrank == 'S1'].sort_values('relab',ascending=False)
                kvals = (df.relab/df.relab.sum()).sort_values(ascending=False).values[:len(vals)]
                # Normalize the distance here 
                # Todo 
                
                # kvals = df.relab[:len(vals)].values

                # wrangle lists
                X = np.array(list(zip(kvals,vals))).T

                # calculate dist
                dist = (ssd.pdist(X,'cityblock'))
                distlist.append(dist[0])
                val_list.append(vals)
                kval_list.append(kvals)

        return np.array(distlist),val_list,kval_list

# Test
# print(single_krakenset('Haemophilus_Influenzae','2strain'))

# Here we call kraken set and make a dataframe just like the strainr one

In [297]:
base = pathlib.Path('/hdd/compare_strainr')
strains = ['Streptococcus_Pneumoniae','Parabacteroides_Distasonis','Haemophilus_Influenzae','Escherichia_Coli','Staph_Aureus']
nstrains = ['2strain','3strain','4strain']
set_of_sets2= {}
set_of_sets3= {}
vlist=[]
klist=[]
for s in strains[:]:
        for n in nstrains[0:1]:
                a,b,c = single_krakenset(s,n)
                set_of_sets2[s] = a
                vlist.append(b)
                klist.append(c)
for s in strains[:]:
        for n in nstrains[1:2]:
                a,b,c = single_krakenset(s,n)
                set_of_sets3[s] = a
                vlist.append(b)
                klist.append(c)
for s in strains[:]:
        for n in nstrains[2:3]:
                a,b,c = single_krakenset(s,n)
                set_of_sets4[s] = a
                vlist.append(b)
                klist.append(c)
kf2 = pd.DataFrame(set_of_sets2)
kf3 = pd.DataFrame(set_of_sets3)
kf4 = pd.DataFrame(set_of_sets4)

In [298]:
kf2['nstrain'] = 2
kf3['nstrain'] = 3
kf4['nstrain'] = 4
kf23 = pd.concat([kf2,kf3,kf4])

# Linreg

In [215]:
klist2 = [iii for i in klist for ii in i for iii in ii]
len(klist2)
sns.set_theme(context='paper',style='white')
len(blist2)
len(vlist2)
dd = dict(Kraken2=klist2,Strainr=blist2,truth=vlist2)
rf = pd.DataFrame(dd)
rrf = rf.melt(value_vars=['Kraken2','Strainr',],id_vars='truth')
g = sns.lmplot(x='value',y='truth',data=rrf,hue='variable',hue_order=['Strainr','Kraken2'])
g.set(xlabel='Predicted Abundance',ylabel='True Abundance',)


# Here we plot a linear regression

In [197]:
from sklearn.linear_model import LinearRegression
sns.set_style('ticks')

# Build list
vlist2 = [iii for i in vlist for ii in i for iii in ii]
blist2 = [iii for i in blist for ii in i for iii in ii]
                        
# Plot
rr = sns.regplot(x=vlist2,y=blist2,)
rr.set(title="Rel Abundance accuracy",xlabel='xlabel',ylabel='ylabel')
sns.despine(trim=True,offset=0,left=0,bottom=0)

# sklearn r2
X = np.array(list(zip(vlist2,klist2)))[:,0]
X = X.reshape((len(X),1))
y = np.array(list(zip(vlist2,klist2)))[:,1]
y = y.reshape((len(y),1))
reg = LinearRegression().fit(y,X)
reg.score(y,X)
reg.coef_
# reg.intercept_


array([[0.45470585]])

# Here we merge the Strainr and Kraken2 dataframes and plot the comparison

In [299]:
sns.set_theme(context='paper',style='white')
kf23['method'] = 'Kraken2'
df23['method'] = 'Strainr'
both = pd.concat([df23,kf23])
# fig,ax = plt.subplots(figsize=(10,4))
# sns.relplot(x='Escherichia_Coli',y='Haemophilus_Influenzae',data=both,hue='method')
melt_vars = ['Streptococcus_Pneumoniae', 'Parabacteroides_Distasonis', 'Haemophilus_Influenzae', 'Escherichia_Coli', 'Staph_Aureus']
melted = both.melt(value_vars=melt_vars,id_vars=['method','nstrain'])
# [' '.join(i.split('_')) for i in melted.columns]
melted.variable = melted.variable.str.replace('_',' ')
melted
g = sns.catplot(kind='bar',x='variable',y='value',hue='method',data=melted,ci=None,row='nstrain',aspect=3,height=3)
# g.set(title='Manhattan distance from True Abundance profile',ylabel='L1 distance',xlabel='Species',ylim=(0,1))
# plt.xticks(rotation=40)
# fig.savefig('results-l1dist.png')

( g.set_axis_labels("", "L1 Distance")
  .set_xticklabels(["S. Pneumoniae", "P. Distasonis", "H. Influenzae","E. Coli","S. Aureus"])
  .set_titles("Number of strains = {row_name} ")
  .set(ylim=(0, 1))
  .despine(left=False) )  
# g.savefig('results-facet234.png')

In [280]:
kf23

,Streptococcus_Pneumoniae,Parabacteroides_Distasonis,Haemophilus_Influenzae,Escherichia_Coli,Staph_Aureus,nstrain,method
0,0.469725,0.779272,0.639359,0.595528,0.873815,2,Kraken2
1,0.675214,0.071853,0.538153,0.204994,0.600760,2,Kraken2
2,0.358323,0.531328,0.564005,0.687500,0.647041,2,Kraken2
3,0.109577,0.046636,0.500000,0.634441,0.505208,2,Kraken2
4,0.716867,0.139162,0.555948,0.564935,0.637037,2,Kraken2
5,0.698276,0.258878,0.550661,0.811321,0.518634,2,Kraken2
6,0.601253,0.136946,0.547101,0.446004,0.133019,2,Kraken2
7,0.645714,0.013628,0.548173,0.579256,0.630299,2,Kraken2
8,0.877723,0.014947,0.632848,0.683333,0.546584,2,Kraken2
9,0.692878,0.456298,0.544521,0.666667,0.597865,2,Kraken2


# this is some seaborn code unrelated

In [165]:
f = plt.figure(figsize=(6, 6))
gs = f.add_gridspec(2, 2)
def sinplot(flip=1):
    x = np.linspace(0, 14, 100)
    for i in range(1, 7):
        plt.plot(x, np.sin(x + i * .5) * (7 - i) * flip)
with sns.axes_style("darkgrid"):
    ax = f.add_subplot(gs[0, 0])
    sinplot()

with sns.axes_style("white"):
    ax = f.add_subplot(gs[0, 1])
    sinplot()

with sns.axes_style("ticks"):
    ax = f.add_subplot(gs[1, 0])
    sinplot()

with sns.axes_style("whitegrid"):
    ax = f.add_subplot(gs[1, 1])
    sinplot()

f.tight_layout()

In [308]:
import seaborn as sns
sns.set_theme(style="darkgrid")

tips = sns.load_dataset("tips")
g = sns.jointplot(x="total_bill", y="tip", data=tips,
                  kind="reg", 
                  truncate=False,
                  xlim=(0, 60), 
                  ylim=(0, 12),
                  color="m", 
                  height=7
                  )

In [315]:
import seaborn as sns
sns.set_theme(style="white")

# Load the example mpg dataset
mpg = sns.load_dataset("mpg")

# Plot miles per gallon against horsepower with other semantics
sns.relplot(x="horsepower", y="mpg", 
            hue="origin", 
            size="weight",
            sizes=(40, 400), 
            alpha=.5,
            palette="muted",
            height=6, 
            data=mpg
            )


In [20]:
import seaborn as sns

sns.set_theme(style="dark")
flights = sns.load_dataset("flights")

# Plot each year's time series in its own facet
g = sns.relplot(
    data=flights,
    x="month", y="passengers", col="year", hue="year",
    kind="line", palette="crest", linewidth=4, 
    zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for year, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.8, .85, year, 
            # transform=ax.transAxes, 
            fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=flights, x="month", y="passengers", units="year",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])

# # Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("", "Passengers")
g.tight_layout()

In [23]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 15))

# Load the example car crash dataset
crashes = sns.load_dataset("car_crashes").sort_values("total", ascending=False)

# Plot the total crashes
sns.set_color_codes("pastel")
sns.barplot(x="total", y="abbrev", data=crashes,
            label="Total", color="b")

# Plot the crashes where alcohol was involved
sns.set_color_codes("muted")
sns.barplot(x="alcohol", y="abbrev", data=crashes,
            label="Alcohol-involved", color="b")

# # Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 24), ylabel="",
       xlabel="Automobile collisions per billion miles")
sns.despine(left=True, bottom=True)

In [49]:
import seaborn as sns
sns.set_theme(style="whitegrid")

# Load the brain networks dataset, select subset, and collapse the multi-index
df = sns.load_dataset("brain_networks", 
                      header=[0, 1, 2],
                      index_col=0
                      )
# df

used_networks = [1, 5, 6, 7, 8, 12, 13, 17]
used_columns = (df.columns
                  .get_level_values("network")
                  .astype(int)
                  .isin(used_networks))
df = df.loc[:, used_columns]


df.columns = df.columns.map("-".join)
# df

# # Compute a correlation matrix and convert to long-form
corr_mat = df.corr().stack().reset_index(name="correlation")
corr_mat

# # Draw each cell as a scatter point with varying size and color
g = sns.relplot(
    data=corr_mat,
    x="level_0", y="level_1", 
    hue="correlation", size="correlation",
    palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
    height=10, sizes=(50, 250), size_norm=(-.2, .8),
)

# # Tweak the figure to finalize
g.set(xlabel="", ylabel="", aspect="equal")
g.despine(left=True, bottom=True)
g.ax.margins(.02)
for label in g.ax.get_xticklabels():
    label.set_rotation(90)
for artist in g.legend.legendHandles:
    artist.set_edgecolor(".7")